In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Directory containing the model output files
data_directory = r"C:\Users\schildress\OneDrive - Metropolitan Transportation Commission\Documents\test_outputs"

print(f"Analyzing files in: {data_directory}")
print("="*80)

Analyzing files in: C:\Users\schildress\OneDrive - Metropolitan Transportation Commission\Documents\test_outputs


In [2]:
def get_file_info(file_path: Path) -> Dict:
    """
    Analyze a single file and return its schema information
    """
    file_info = {
        'filename': file_path.name,
        'file_size_mb': round(file_path.stat().st_size / (1024 * 1024), 2),
        'file_extension': file_path.suffix.lower(),
        'columns': [],
        'data_types': {},
        'shape': None,
        'sample_data': None,
        'error': None
    }
    
    try:
        # Determine file type and read accordingly
        if file_path.suffix.lower() == '.csv':
            # Read just the first few rows to get schema
            df = pd.read_csv(file_path, nrows=1000)
        elif file_path.suffix.lower() in ['.xlsx', '.xls']:
            df = pd.read_excel(file_path, nrows=1000)
        elif file_path.suffix.lower() == '.parquet':
            df = pd.read_parquet(file_path)
            # For parquet, just take first 1000 rows for sample
            df = df.head(1000)
        elif file_path.suffix.lower() in ['.txt', '.tsv']:
            # Try tab-separated first, then comma
            try:
                df = pd.read_csv(file_path, sep='\t', nrows=1000)
            except:
                df = pd.read_csv(file_path, nrows=1000)
        else:
            # Try to read as CSV by default
            df = pd.read_csv(file_path, nrows=1000)
        
        # Extract schema information
        file_info['columns'] = list(df.columns)
        file_info['data_types'] = {col: str(dtype) for col, dtype in df.dtypes.items()}
        file_info['shape'] = df.shape
        file_info['sample_data'] = df.head(3).to_dict('records')
        
    except Exception as e:
        file_info['error'] = str(e)
    
    return file_info

In [3]:
def analyze_directory_schema(directory_path: str) -> Dict:
    """
    Analyze all files in the directory and return comprehensive schema information
    """
    data_dir = Path(directory_path)
    
    if not data_dir.exists():
        return {'error': f"Directory {directory_path} does not exist"}
    
    # Find all data files (common formats)
    data_extensions = {'.csv', '.xlsx', '.xls', '.parquet', '.txt', '.tsv', '.json'}
    data_files = []
    
    for ext in data_extensions:
        data_files.extend(list(data_dir.glob(f"*{ext}")))
        data_files.extend(list(data_dir.glob(f"**/*{ext}")))  # Include subdirectories
    
    print(f"Found {len(data_files)} data files")
    
    # Analyze each file
    file_schemas = []
    for file_path in data_files:
        print(f"Analyzing: {file_path.name}")
        schema_info = get_file_info(file_path)
        file_schemas.append(schema_info)
    
    return {
        'directory': str(data_dir),
        'total_files': len(data_files),
        'file_schemas': file_schemas
    }

In [8]:
# Run the schema analysis
schema_results = analyze_directory_schema(data_directory)

Found 12 data files
Analyzing: householdData_1.csv
Analyzing: indivTourData_1.csv
Analyzing: indivTripData_1.csv
Analyzing: jointTourData_1.csv
Analyzing: jointTripData_1.csv
Analyzing: personData_1.csv
Analyzing: householdData_1.csv
Analyzing: indivTourData_1.csv
Analyzing: indivTripData_1.csv
Analyzing: jointTourData_1.csv
Analyzing: jointTripData_1.csv
Analyzing: personData_1.csv


In [5]:
def print_schema_summary(results: Dict):
    """
    Print a formatted summary of the schema analysis results
    """
    if 'error' in results:
        print(f"Error: {results['error']}")
        return
    
    print(f"\n📁 DIRECTORY ANALYSIS SUMMARY")
    print(f"Directory: {results['directory']}")
    print(f"Total Files Found: {results['total_files']}")
    print("="*80)
    
    for i, file_info in enumerate(results['file_schemas'], 1):
        print(f"\n📄 FILE {i}: {file_info['filename']}")
        print(f"   Size: {file_info['file_size_mb']} MB")
        print(f"   Extension: {file_info['file_extension']}")
        
        if file_info['error']:
            print(f"   ❌ Error: {file_info['error']}")
            continue
            
        print(f"   Shape: {file_info['shape']} (rows, columns)")
        print(f"   Columns ({len(file_info['columns'])}):")
        
        # Print columns and data types in a nice format
        for col in file_info['columns']:
            dtype = file_info['data_types'].get(col, 'unknown')
            print(f"      • {col:<30} ({dtype})")
        
        # Show sample data if available
        if file_info['sample_data'] and len(file_info['sample_data']) > 0:
            print(f"   Sample Data (first row):")
            first_row = file_info['sample_data'][0]
            for key, value in list(first_row.items())[:5]:  # Show first 5 columns
                print(f"      {key}: {value}")
            if len(first_row) > 5:
                print(f"      ... and {len(first_row)-5} more columns")
        
        print("-"*60)

# Print the results
print_schema_summary(schema_results)


📁 DIRECTORY ANALYSIS SUMMARY
Directory: C:\Users\schildress\OneDrive - Metropolitan Transportation Commission\Documents\test_outputs
Total Files Found: 10

📄 FILE 1: householdData_1.csv
   Size: 7.73 MB
   Extension: .csv
   Shape: (1000, 12) (rows, columns)
   Columns (12):
      • hh_id                          (int64)
      • home_mgra                      (int64)
      • income                         (int64)
      • autos                          (int64)
      • automated_vehicles             (int64)
      • transponder                    (int64)
      • pre_et_cdap_pattern            (object)
      • cdap_pattern                   (object)
      • jtf_choice                     (int64)
      • sampleRate                     (float64)
      • size                           (int64)
      • workers                        (int64)
   Sample Data (first row):
      hh_id: 1581987
      home_mgra: 1
      income: 44257
      autos: 2
      automated_vehicles: 0
      ... and 7 more col

In [6]:
# Create a consolidated schema DataFrame for easy export/analysis
def create_schema_dataframe(results: Dict) -> pd.DataFrame:
    """
    Create a consolidated DataFrame with all schema information
    """
    schema_rows = []
    
    for file_info in results['file_schemas']:
        if file_info['error']:
            schema_rows.append({
                'filename': file_info['filename'],
                'file_size_mb': file_info['file_size_mb'],
                'file_extension': file_info['file_extension'],
                'column_name': None,
                'data_type': None,
                'error': file_info['error'],
                'total_columns': None,
                'total_rows': None
            })
        else:
            for col in file_info['columns']:
                schema_rows.append({
                    'filename': file_info['filename'],
                    'file_size_mb': file_info['file_size_mb'],
                    'file_extension': file_info['file_extension'],
                    'column_name': col,
                    'data_type': file_info['data_types'].get(col, 'unknown'),
                    'error': None,
                    'total_columns': len(file_info['columns']),
                    'total_rows': file_info['shape'][0] if file_info['shape'] else None
                })
    
    return pd.DataFrame(schema_rows)

# Create the consolidated schema DataFrame
schema_df = create_schema_dataframe(schema_results)
print(f"\n📊 CONSOLIDATED SCHEMA DATA")
print(f"Total schema entries: {len(schema_df)}")
print("\nFirst 10 entries:")
print(schema_df.head(10))


📊 CONSOLIDATED SCHEMA DATA
Total schema entries: 310

First 10 entries:
              filename  file_size_mb file_extension          column_name  \
0  householdData_1.csv          7.73           .csv                hh_id   
1  householdData_1.csv          7.73           .csv            home_mgra   
2  householdData_1.csv          7.73           .csv               income   
3  householdData_1.csv          7.73           .csv                autos   
4  householdData_1.csv          7.73           .csv   automated_vehicles   
5  householdData_1.csv          7.73           .csv          transponder   
6  householdData_1.csv          7.73           .csv  pre_et_cdap_pattern   
7  householdData_1.csv          7.73           .csv         cdap_pattern   
8  householdData_1.csv          7.73           .csv           jtf_choice   
9  householdData_1.csv          7.73           .csv           sampleRate   

  data_type error  total_columns  total_rows  
0     int64  None             12        100

In [7]:
# Export schema information to CSV for future reference
output_file = "ctramp_2023_data_schema.csv"
schema_df.to_csv(output_file, index=False)
print(f"✅ Schema information exported to: {output_file}")

# Show summary statistics
print(f"\n📈 SUMMARY STATISTICS")
print(f"Unique files analyzed: {schema_df['filename'].nunique()}")
print(f"Total data types found: {schema_df['data_type'].nunique()}")
print(f"File types: {schema_df['file_extension'].value_counts().to_dict()}")

# Show most common column names (could indicate standard fields)
print(f"\n🏷️ MOST COMMON COLUMN NAMES:")
common_columns = schema_df['column_name'].value_counts().head(10)
for col_name, count in common_columns.items():
    if pd.notna(col_name):
        print(f"   {col_name}: appears in {count} files")

# Show data type distribution
print(f"\n📊 DATA TYPE DISTRIBUTION:")
dtype_counts = schema_df['data_type'].value_counts()
for dtype, count in dtype_counts.items():
    if pd.notna(dtype):
        print(f"   {dtype}: {count} columns")

✅ Schema information exported to: ctramp_2023_data_schema.csv

📈 SUMMARY STATISTICS
Unique files analyzed: 5
Total data types found: 3
File types: {'.csv': 310}

🏷️ MOST COMMON COLUMN NAMES:
   hh_id: appears in 10 files
   sampleRate: appears in 10 files
   dest_mgra: appears in 8 files
   orig_mgra: appears in 8 files
   tour_purpose: appears in 8 files
   tour_id: appears in 8 files
   avAvailable: appears in 8 files
   tour_mode: appears in 8 files
   prob_3: appears in 4 files
   prob_2: appears in 4 files

📊 DATA TYPE DISTRIBUTION:
   float64: 164 columns
   int64: 120 columns
   object: 26 columns


# CTRAMP 2023 Model Output Schema Analysis

This notebook analyzes the data schema of Bay Area residential travel demand model output files from 2023. The script:

1. **Scans the specified directory** for common data file formats (CSV, Excel, Parquet, etc.)
2. **Extracts schema information** including:
   - Column names and data types
   - File sizes and shapes
   - Sample data from each file
3. **Creates a consolidated summary** of all schemas
4. **Exports results** to CSV for further analysis

## Key Features:
- Handles multiple file formats automatically
- Provides detailed error reporting for problematic files
- Creates summary statistics about common columns and data types
- Exports all findings to a structured CSV file

## Usage:
Simply update the `data_directory` variable in the first cell to point to your model output folder and run all cells.

In [9]:
# CTRAMP Documentation Integration
# Define expected schemas based on official CTRAMP documentation

CTRAMP_FILE_SCHEMAS = {
    'accessibilities.csv': {
        'description': 'Various accessibility measures by microzone',
        'expected_fields': ['mgra'],  # Basic field, actual schema varies
        'file_type': 'accessibility'
    },
    
    'aoResults_pre.csv': {
        'description': 'Household auto ownership before work and school location choice',
        'expected_fields': ['hh_id'],
        'file_type': 'auto_ownership'
    },
    
    'aoResults.csv': {
        'description': 'Household auto ownership after work and school location choice',
        'expected_fields': ['hh_id'],
        'file_type': 'auto_ownership'
    },
    
    'householdData': {
        'description': 'Household level model results',
        'expected_fields': ['hh_id', 'transponder', 'cdap_pattern', 'jtf_choice'],
        'file_type': 'household'
    },
    
    'personData': {
        'description': 'Person level model results',
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'value_of_time', 
                          'transitSubsidy_choice', 'transitSubsidy_percent', 'transitPass_choice',
                          'activity_pattern', 'imf_choice', 'inmf_choice', 'fp_choice', 
                          'reimb_pct', 'workDCLogsum', 'schoolDCLogsum'],
        'file_type': 'person'
    },
    
    'wsLocResults': {
        'description': 'Work and school location choice results',
        'expected_fields': ['hh_id', 'person_id', 'EmploymentCategory', 'StudentCategory', 
                          'WorkSegment', 'SchoolSegment', 'WorkLocation', 'WorkLocationDistance',
                          'WorkLocationLogsum', 'SchoolLocation', 'SchoolLocationDistance', 'SchoolLocationLogsum'],
        'file_type': 'location_choice'
    },
    
    'indivTourData': {
        'description': 'Individual Tours - one row per individual tour',
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'person_type', 'tour_id', 
                          'tour_category', 'tour_purpose', 'orig_mgra', 'dest_mgra', 
                          'start_period', 'end_period', 'tour_mode', 'tour_distance', 
                          'tour_time', 'atWork_freq', 'num_ob_stops', 'num_ib_stops',
                          'out_btap', 'out_atap', 'in_btap', 'in_atap', 'out_set', 'in_set',
                          'sampleRate', 'dcLogsum'] + [f'util_{i}' for i in range(1,18)] + [f'prob_{i}' for i in range(1,18)],
        'file_type': 'individual_tour'
    },
    
    'indivTripData': {
        'description': 'Individual Trips - one row per individual trip',
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
                          'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
                          'parking_mgra', 'stop_period', 'trip_mode', 'trip_board_tap', 
                          'trip_alight_tap', 'tour_mode', 'set', 'sampleRate', 'TRIP_TIME', 
                          'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'individual_trip'
    },
    
    'jointTourData': {
        'description': 'Joint Tours - one row per joint tour (includes all travelers)',
        'expected_fields': ['hh_id', 'tour_id', 'tour_category', 'tour_purpose', 'tour_composition',
                          'tour_participants', 'orig_mgra', 'dest_mgra', 'start_period', 'end_period',
                          'tour_mode', 'tour_distance', 'tour_time', 'num_ob_stops', 'num_ib_stops',
                          'out_btap', 'out_atap', 'in_btap', 'in_atap', 'out_set', 'in_set',
                          'sampleRate', 'dcLogsum'] + [f'util_{i}' for i in range(1,18)] + [f'prob_{i}' for i in range(1,18)],
        'file_type': 'joint_tour'
    },
    
    'jointTripData': {
        'description': 'Joint Trips - one row per joint trip',
        'expected_fields': ['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose', 'orig_purpose',
                          'dest_purpose', 'orig_mgra', 'dest_mgra', 'parking_mgra', 'stop_period',
                          'trip_mode', 'num_participants', 'trip_board_tap', 'trip_alight_tap',
                          'tour_mode', 'set', 'sampleRate', 'TRIP_TIME', 'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'joint_trip'
    },
    
    'indivTripDataResim': {
        'description': 'Resimulated Transit Trips',
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
                          'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
                          'parking_mgra', 'stop_period', 'trip_mode', 'trip_board_tap', 
                          'trip_alight_tap', 'tour_mode', 'set', 'sampleRate', 'resimulatedTrip',
                          'TRIP_TIME', 'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'resimulated_trip'
    },
    
    'unconstrainedPNRDemand': {
        'description': 'Unconstrained Parking Demand by TAP and time period',
        'expected_fields': ['TAP', 'CAPACITY', 'TOT_ARRIVALS'],  # Plus time period columns
        'file_type': 'parking_demand'
    },
    
    'constrainedPNRDemand': {
        'description': 'Constrained Parking Demand by TAP and time period', 
        'expected_fields': ['TAP', 'CAPACITY', 'TOT_ARRIVALS'],  # Plus time period columns
        'file_type': 'parking_demand'
    }
}

# Mode codes mapping
MODE_CODES = {
    1: 'DRIVEALONEFREE', 2: 'DRIVEALONEPAY', 3: 'SHARED2GP', 4: 'SHARED2HOV', 
    5: 'SHARED2PAY', 6: 'SHARED3GP', 7: 'SHARED3HOV', 8: 'SHARED3PAY',
    9: 'WALK', 10: 'BIKE', 11: 'WALK_SET', 12: 'PNR_SET', 13: 'KNR_PERS',
    14: 'KNR_TNC', 15: 'TAXI', 16: 'TNC', 17: 'SCHBUS'
}

# Time period mapping
TIME_PERIODS = {
    1: '03:00 AM to 05:00 AM', 2: '05:00 AM to 05:30 AM', 3: '05:30 AM to 06:00 AM',
    4: '06:00 AM to 06:30 AM', 5: '06:30 AM to 07:00 AM', 6: '07:00 AM to 07:30 AM',
    7: '07:30 AM to 08:00 AM', 8: '08:00 AM to 08:30 AM', 9: '08:30 AM to 09:00 AM',
    10: '09:00 AM to 09:30 AM', 11: '09:30 AM to 10:00 AM', 12: '10:00 AM to 10:30 AM',
    13: '10:30 AM to 11:00 AM', 14: '11:00 AM to 11:30 AM', 15: '11:30 AM to 12:00 PM',
    16: '12:00 PM to 12:30 PM', 17: '12:30 PM to 01:00 PM', 18: '01:00 PM to 01:30 PM',
    19: '01:30 PM to 02:00 PM', 20: '02:00 PM to 02:30 PM', 21: '02:30 PM to 03:00 PM',
    22: '03:00 PM to 03:30 PM', 23: '03:30 PM to 04:00 PM', 24: '04:00 PM to 04:30 PM',
    25: '04:30 PM to 05:00 PM', 26: '05:00 PM to 05:30 PM', 27: '05:30 PM to 06:00 PM',
    28: '06:00 PM to 06:30 PM', 29: '06:30 PM to 07:00 PM', 30: '07:00 PM to 07:30 PM',
    31: '07:30 PM to 08:00 PM', 32: '08:00 PM to 08:30 PM', 33: '08:30 PM to 09:00 PM',
    34: '09:00 PM to 09:30 PM', 35: '09:30 PM to 10:00 PM', 36: '10:00 PM to 10:30 PM',
    37: '10:30 PM to 11:00 PM', 38: '11:00 PM to 11:30 PM', 39: '11:30 PM to 12:00 AM',
    40: '12:00 AM to 03:00 AM'
}

print("✅ CTRAMP documentation schemas loaded!")
print(f"📋 Defined schemas for {len(CTRAMP_FILE_SCHEMAS)} file types")
print(f"🚗 Mode codes: {len(MODE_CODES)} modes defined")  
print(f"⏰ Time periods: {len(TIME_PERIODS)} periods defined")

✅ CTRAMP documentation schemas loaded!
📋 Defined schemas for 13 file types
🚗 Mode codes: 17 modes defined
⏰ Time periods: 40 periods defined


In [ ]:
# Enhanced CTRAMP Documentation for Analysis and Summarization
# This comprehensive documentation will help with future analysis tasks

ENHANCED_CTRAMP_SCHEMAS = {
    'accessibilities.csv': {
        'description': 'Zone-level accessibility measures by microzone (MGRA)',
        'primary_key': ['mgra'],
        'expected_fields': ['mgra'],  # Will be updated based on actual data
        'file_type': 'accessibility',
        'record_level': 'zone',
        'typical_size': '10K-50K rows (one per MGRA)',
        'analysis_notes': {
            'purpose': 'Accessibility to jobs, schools, other activities by mode and time period',
            'common_summaries': ['accessibility by county', 'accessibility by income', 'mode-specific accessibility'],
            'key_fields_for_analysis': ['mgra', 'auto_accessibility', 'transit_accessibility'],
            'geographic_aggregation': 'Can aggregate to TAZ, county, or equity zones using MGRA lookups'
        }
    },
    
    'aoResults_pre.csv': {
        'description': 'Household auto ownership before work/school location choice',
        'primary_key': ['hh_id'],
        'expected_fields': ['hh_id'],  # Will be updated
        'file_type': 'auto_ownership',
        'record_level': 'household', 
        'typical_size': '500K-1M rows (one per household)',
        'analysis_notes': {
            'purpose': 'Pre-location choice vehicle ownership model results',
            'common_summaries': ['vehicles per household by income', 'ownership rates by area type'],
            'key_fields_for_analysis': ['hh_id', 'vehicles', 'income_category'],
            'comparison_file': 'aoResults.csv shows post-location choice results'
        }
    },
    
    'aoResults.csv': {
        'description': 'Household auto ownership after work/school location choice',
        'primary_key': ['hh_id'],
        'expected_fields': ['hh_id'],  # Will be updated
        'file_type': 'auto_ownership',
        'record_level': 'household',
        'typical_size': '500K-1M rows (one per household)', 
        'analysis_notes': {
            'purpose': 'Final vehicle ownership after location choices',
            'common_summaries': ['change in ownership due to location', 'final ownership distribution'],
            'key_fields_for_analysis': ['hh_id', 'vehicles', 'change_from_pre'],
            'comparison_file': 'Compare with aoResults_pre.csv to see location impact'
        }
    },
    
    'householdData': {
        'description': 'Household characteristics and choice model results',
        'primary_key': ['hh_id'],
        'expected_fields': ['hh_id', 'transponder', 'cdap_pattern', 'jtf_choice'],  # Will expand
        'file_type': 'household',
        'record_level': 'household',
        'typical_size': '500K-1M rows (one per household)',
        'analysis_notes': {
            'purpose': 'Core household data with demographics and major choice model results',
            'common_summaries': ['household size distribution', 'income distribution', 'CDAP patterns', 'joint tour frequency'],
            'key_fields_for_analysis': ['hh_id', 'hhsize', 'income', 'workers', 'cdap_pattern', 'vehicles'],
            'geographic_aggregation': 'Use home_mgra to aggregate spatially',
            'model_validation': 'Compare CDAP patterns to survey data, check income distribution'
        }
    },
    
    'personData': {
        'description': 'Person characteristics and individual choice model results',
        'primary_key': ['hh_id', 'person_id'],
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'value_of_time', 
                          'transitSubsidy_choice', 'transitSubsidy_percent', 'transitPass_choice',
                          'activity_pattern', 'imf_choice', 'inmf_choice', 'fp_choice', 
                          'reimb_pct', 'workDCLogsum', 'schoolDCLogsum'],
        'file_type': 'person',
        'record_level': 'person',
        'typical_size': '1M-2M rows (2-3 persons per household average)',
        'analysis_notes': {
            'purpose': 'Individual person attributes and activity/tour generation results',
            'common_summaries': ['person type distribution', 'activity patterns by demographics', 'value of time distribution'],
            'key_fields_for_analysis': ['person_type', 'age', 'gender', 'worker_status', 'activity_pattern', 'value_of_time'],
            'model_validation': 'Check activity pattern rates vs survey, validate value of time by income',
            'join_keys': 'Join to householdData on hh_id for household characteristics'
        }
    },
    
    'wsLocResults': {
        'description': 'Work and school location choice model results',
        'primary_key': ['hh_id', 'person_id'],
        'expected_fields': ['hh_id', 'person_id', 'EmploymentCategory', 'StudentCategory', 
                          'WorkSegment', 'SchoolSegment', 'WorkLocation', 'WorkLocationDistance',
                          'WorkLocationLogsum', 'SchoolLocation', 'SchoolLocationDistance', 'SchoolLocationLogsum'],
        'file_type': 'location_choice',
        'record_level': 'person',
        'typical_size': '500K-1M rows (workers and students only)',
        'analysis_notes': {
            'purpose': 'Workplace and school location choices with distances and logsums',
            'common_summaries': ['commute distance distribution', 'jobs-housing balance', 'school enrollment by district'],
            'key_fields_for_analysis': ['WorkLocation', 'WorkLocationDistance', 'SchoolLocation', 'employment_category'],
            'geographic_analysis': 'Map work/school flows, calculate commute patterns by county',
            'model_validation': 'Compare commute distances to survey data'
        }
    },
    
    'indivTourData': {
        'description': 'Individual tours with full mode choice model results',
        'primary_key': ['hh_id', 'person_id', 'tour_id'],
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'person_type', 'tour_id', 
                          'tour_category', 'tour_purpose', 'orig_mgra', 'dest_mgra', 
                          'start_period', 'end_period', 'tour_mode', 'tour_distance', 
                          'tour_time', 'atWork_freq', 'num_ob_stops', 'num_ib_stops',
                          'out_btap', 'out_atap', 'in_btap', 'in_atap', 'out_set', 'in_set',
                          'sampleRate', 'dcLogsum'] + [f'util_{i}' for i in range(1,18)] + [f'prob_{i}' for i in range(1,18)],
        'file_type': 'individual_tour',
        'record_level': 'tour',
        'typical_size': '2M-5M rows (2-3 tours per person average)',
        'analysis_notes': {
            'purpose': 'Individual daily tours with mode choice utilities and probabilities',
            'common_summaries': ['mode share by purpose', 'tour generation rates', 'time-of-day patterns', 'tour length distribution'],
            'key_fields_for_analysis': ['tour_purpose', 'tour_mode', 'start_period', 'end_period', 'tour_distance', 'orig_mgra', 'dest_mgra'],
            'mode_choice_analysis': 'Use util_* and prob_* fields for mode choice model validation',
            'time_analysis': 'start_period and end_period for departure/arrival time analysis',
            'geographic_analysis': 'Origin-destination flows using orig_mgra and dest_mgra',
            'transit_analysis': 'Use *_btap and *_atap fields for transit boarding/alighting analysis'
        }
    },
    
    'indivTripData': {
        'description': 'Individual trips (tour components) with detailed attributes',
        'primary_key': ['hh_id', 'person_id', 'tour_id', 'stop_id'],
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
                          'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
                          'parking_mgra', 'stop_period', 'trip_mode', 'trip_board_tap', 
                          'trip_alight_tap', 'tour_mode', 'set', 'sampleRate', 'TRIP_TIME', 
                          'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'individual_trip',
        'record_level': 'trip',
        'typical_size': '5M-15M rows (multiple trips per tour)',
        'analysis_notes': {
            'purpose': 'Individual trip segments with time, distance, and cost',
            'common_summaries': ['trip mode share', 'trip length distribution', 'VMT by purpose', 'trip timing patterns'],
            'key_fields_for_analysis': ['trip_mode', 'orig_purpose', 'dest_purpose', 'TRIP_DISTANCE', 'TRIP_TIME', 'stop_period'],
            'vmt_calculation': 'Sum TRIP_DISTANCE by trip_mode for vehicle miles traveled',
            'transit_analysis': 'trip_board_tap and trip_alight_tap for detailed transit flows',
            'time_analysis': 'stop_period for hourly travel patterns',
            'cost_analysis': 'TRIP_COST for travel cost distribution by income/mode'
        }
    },
    
    'jointTourData': {
        'description': 'Joint household tours (multiple participants)',
        'primary_key': ['hh_id', 'tour_id'],
        'expected_fields': ['hh_id', 'tour_id', 'tour_category', 'tour_purpose', 'tour_composition',
                          'tour_participants', 'orig_mgra', 'dest_mgra', 'start_period', 'end_period',
                          'tour_mode', 'tour_distance', 'tour_time', 'num_ob_stops', 'num_ib_stops',
                          'out_btap', 'out_atap', 'in_btap', 'in_atap', 'out_set', 'in_set',
                          'sampleRate', 'dcLogsum'] + [f'util_{i}' for i in range(1,18)] + [f'prob_{i}' for i in range(1,18)],
        'file_type': 'joint_tour',
        'record_level': 'joint_tour',
        'typical_size': '500K-2M rows (fewer than individual tours)',
        'analysis_notes': {
            'purpose': 'Household joint travel with multiple participants per tour',
            'common_summaries': ['joint travel rates', 'joint tour purposes', 'household coordination patterns'],
            'key_fields_for_analysis': ['tour_purpose', 'tour_composition', 'tour_participants', 'tour_mode'],
            'occupancy_analysis': 'Use tour_participants to understand vehicle occupancy',
            'household_analysis': 'Compare joint vs individual travel patterns by household type'
        }
    },
    
    'jointTripData': {
        'description': 'Joint household trips (components of joint tours)',  
        'primary_key': ['hh_id', 'tour_id', 'stop_id'],
        'expected_fields': ['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose', 'orig_purpose',
                          'dest_purpose', 'orig_mgra', 'dest_mgra', 'parking_mgra', 'stop_period',
                          'trip_mode', 'num_participants', 'trip_board_tap', 'trip_alight_tap',
                          'tour_mode', 'set', 'sampleRate', 'TRIP_TIME', 'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'joint_trip',
        'record_level': 'joint_trip',
        'typical_size': '1M-5M rows (trips within joint tours)',
        'analysis_notes': {
            'purpose': 'Joint trip segments with participant counts',
            'common_summaries': ['joint trip VMT', 'vehicle occupancy rates', 'joint trip destinations'],
            'key_fields_for_analysis': ['trip_mode', 'num_participants', 'TRIP_DISTANCE', 'orig_purpose', 'dest_purpose'],
            'occupancy_calculation': 'Use num_participants for accurate vehicle occupancy analysis',
            'vmt_adjustment': 'TRIP_DISTANCE represents vehicle miles, not person miles'
        }
    },
    
    'indivTripDataResim': {
        'description': 'Resimulated transit trips with additional transit assignment data',
        'primary_key': ['hh_id', 'person_id', 'tour_id', 'stop_id'],
        'expected_fields': ['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
                          'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
                          'parking_mgra', 'stop_period', 'trip_mode', 'trip_board_tap', 
                          'trip_alight_tap', 'tour_mode', 'set', 'sampleRate', 'resimulatedTrip',
                          'TRIP_TIME', 'TRIP_DISTANCE', 'TRIP_COST'],
        'file_type': 'resimulated_trip',
        'record_level': 'trip',
        'typical_size': 'Subset of indivTripData (transit trips only)',
        'analysis_notes': {
            'purpose': 'Transit trips with updated paths from transit assignment feedback',
            'common_summaries': ['transit mode share', 'transit boarding patterns', 'route choice validation'],
            'key_fields_for_analysis': ['resimulatedTrip', 'trip_board_tap', 'trip_alight_tap', 'set'],
            'transit_validation': 'Compare with original indivTripData to see assignment impacts',
            'path_analysis': 'Use set field to analyze transit route choices'
        }
    }
}

print("✅ Enhanced CTRAMP documentation loaded for analysis!")
print(f"📊 Detailed schemas with analysis guidance for {len(ENHANCED_CTRAMP_SCHEMAS)} file types")

In [ ]:
# Analysis Helper Functions for Future Summarization Tasks

def get_analysis_guidance(file_type: str) -> dict:
    """
    Get specific analysis guidance for a CTRAMP file type
    """
    for pattern, schema in ENHANCED_CTRAMP_SCHEMAS.items():
        if pattern.replace('_[iteration]', '').replace('[iteration]', '') in file_type.lower():
            return schema.get('analysis_notes', {})
    return {}

def get_common_summaries(filename: str) -> list:
    """
    Get list of common summary analyses for a file type
    """
    guidance = get_analysis_guidance(filename)
    return guidance.get('common_summaries', [])

def get_key_analysis_fields(filename: str) -> list:
    """
    Get the most important fields for analysis in a file
    """
    guidance = get_analysis_guidance(filename)
    return guidance.get('key_fields_for_analysis', [])

def estimate_expected_size(filename: str) -> str:
    """
    Get expected file size/record count for validation
    """
    for pattern, schema in ENHANCED_CTRAMP_SCHEMAS.items():
        if pattern.replace('_[iteration]', '').replace('[iteration]', '') in filename.lower():
            return schema.get('typical_size', 'Size varies')
    return 'Unknown size'

# Summary Analysis Templates
SUMMARY_TEMPLATES = {
    'mode_share': {
        'description': 'Calculate mode share by various demographics and geography',
        'applicable_files': ['indivTourData', 'indivTripData', 'jointTourData', 'jointTripData'],
        'key_fields': ['tour_mode', 'trip_mode'],
        'groupby_options': ['tour_purpose', 'income_category', 'person_type', 'time_period'],
        'code_template': """
# Mode Share Analysis
mode_share = df.groupby(['groupby_field'])['mode_field'].value_counts(normalize=True).unstack(fill_value=0)
mode_share_pct = mode_share * 100
        """
    },
    
    'vmt_calculation': {
        'description': 'Calculate Vehicle Miles Traveled by various categories',
        'applicable_files': ['indivTripData', 'jointTripData'],
        'key_fields': ['TRIP_DISTANCE', 'trip_mode'],
        'vehicle_modes': [1, 2, 3, 4, 5, 6, 7, 8],  # Drive alone and shared ride modes
        'code_template': """
# VMT Calculation
vehicle_trips = df[df['trip_mode'].isin([1,2,3,4,5,6,7,8])]
vmt_by_purpose = vehicle_trips.groupby('orig_purpose')['TRIP_DISTANCE'].sum()
        """
    },
    
    'time_of_day': {
        'description': 'Analyze departure/arrival time patterns',
        'applicable_files': ['indivTourData', 'indivTripData'],
        'key_fields': ['start_period', 'end_period', 'stop_period'],
        'time_periods': TIME_PERIODS,
        'code_template': """
# Time of Day Analysis
tod_pattern = df.groupby(['time_field']).size()
# Convert period numbers to time labels using TIME_PERIODS dict
        """
    },
    
    'accessibility_summary': {
        'description': 'Summarize accessibility measures by geography and demographics',
        'applicable_files': ['accessibilities.csv'],
        'key_fields': ['mgra'],
        'join_requirements': 'Need MGRA to TAZ/County lookup table',
        'code_template': """
# Accessibility Summary by Geography
# Join with geography lookup: accessibility.merge(mgra_lookup, on='mgra')
acc_by_county = df.groupby('county')[accessibility_fields].mean()
        """
    }
}

def get_summary_template(analysis_type: str) -> dict:
    """
    Get a pre-built analysis template
    """
    return SUMMARY_TEMPLATES.get(analysis_type, {})

def list_applicable_summaries(filename: str) -> list:
    """
    List all applicable summary analyses for a given file
    """
    applicable = []
    for analysis_type, template in SUMMARY_TEMPLATES.items():
        if any(file_pattern in filename for file_pattern in template['applicable_files']):
            applicable.append(analysis_type)
    return applicable

print("🛠️ Analysis helper functions loaded!")
print(f"📝 {len(SUMMARY_TEMPLATES)} pre-built analysis templates available:")
for template_name in SUMMARY_TEMPLATES.keys():
    print(f"   • {template_name}")

# Quick reference for mode codes and time periods
print(f"\n🚗 Mode codes: {len(MODE_CODES)} modes (1=Drive Alone Free, 9=Walk, 11=Walk-Transit, etc.)")
print(f"⏰ Time periods: {len(TIME_PERIODS)} periods (1=3-5am, 8=8-8:30am peak, etc.)")

In [ ]:
# Update validation functions to use enhanced documentation
def validate_with_enhanced_schemas(file_info: dict) -> dict:
    """
    Enhanced validation using the detailed analysis-ready schemas
    """
    filename = file_info['filename']
    
    # Find matching enhanced schema
    enhanced_schema = None
    schema_key = None
    for pattern, schema in ENHANCED_CTRAMP_SCHEMAS.items():
        if pattern.replace('_[iteration]', '').replace('[iteration]', '') in filename.lower():
            enhanced_schema = schema
            schema_key = pattern
            break
    
    validation_result = {
        'filename': filename,
        'identified_type': schema_key,
        'record_level': enhanced_schema.get('record_level', 'unknown') if enhanced_schema else 'unknown',
        'expected_size': enhanced_schema.get('typical_size', 'unknown') if enhanced_schema else 'unknown',
        'primary_keys': enhanced_schema.get('primary_key', []) if enhanced_schema else [],
        'analysis_ready': enhanced_schema is not None,
        'common_analyses': enhanced_schema.get('analysis_notes', {}).get('common_summaries', []) if enhanced_schema else [],
        'key_analysis_fields': enhanced_schema.get('analysis_notes', {}).get('key_fields_for_analysis', []) if enhanced_schema else [],
        'applicable_templates': list_applicable_summaries(filename),
        'field_coverage': 0,
        'missing_critical_fields': [],
        'validation_notes': []
    }
    
    if file_info.get('error'):
        validation_result['validation_notes'].append(f"ERROR: {file_info['error']}")
        return validation_result
    
    actual_columns = set(file_info.get('columns', []))
    
    if enhanced_schema:
        expected_fields = set(enhanced_schema['expected_fields'])
        critical_fields = set(enhanced_schema.get('primary_key', []))
        analysis_fields = set(enhanced_schema.get('analysis_notes', {}).get('key_fields_for_analysis', []))
        
        # Calculate field coverage
        matching_expected = len(expected_fields & actual_columns)
        if len(expected_fields) > 0:
            validation_result['field_coverage'] = (matching_expected / len(expected_fields)) * 100
        
        # Check for missing critical fields
        validation_result['missing_critical_fields'] = list(critical_fields - actual_columns)
        missing_analysis_fields = list(analysis_fields - actual_columns)
        
        # Validation notes
        if validation_result['field_coverage'] >= 90:
            validation_result['validation_notes'].append("✅ Excellent field coverage")
        elif validation_result['field_coverage'] >= 70:
            validation_result['validation_notes'].append("👍 Good field coverage")
        else:
            validation_result['validation_notes'].append("⚠️ Low field coverage - check schema")
            
        if validation_result['missing_critical_fields']:
            validation_result['validation_notes'].append(f"❌ Missing primary keys: {validation_result['missing_critical_fields']}")
            
        if missing_analysis_fields:
            validation_result['validation_notes'].append(f"📊 Missing key analysis fields: {missing_analysis_fields[:3]}")
            
        # Size validation
        if file_info.get('shape'):
            actual_rows = file_info['shape'][0]
            expected_size = enhanced_schema.get('typical_size', '')
            if 'K' in expected_size:
                min_expected = int(expected_size.split('-')[0].replace('K', '')) * 1000
                if actual_rows < min_expected * 0.5:  # Allow 50% tolerance
                    validation_result['validation_notes'].append(f"⚠️ Low row count: {actual_rows:,} vs expected {expected_size}")
    else:
        validation_result['validation_notes'].append("❓ File type not in enhanced documentation")
    
    return validation_result

# Function to show analysis recommendations for a file
def show_analysis_recommendations(filename: str, file_columns: list = None):
    """
    Show specific analysis recommendations for a file
    """
    print(f"\n🎯 ANALYSIS RECOMMENDATIONS FOR: {filename}")
    print("="*60)
    
    # Get guidance
    guidance = get_analysis_guidance(filename)
    if not guidance:
        print("❓ No specific guidance available for this file type")
        return
    
    print(f"📝 Purpose: {guidance.get('purpose', 'No description')}")
    
    print(f"\n🔍 Common Summaries:")
    for summary in guidance.get('common_summaries', []):
        print(f"   • {summary}")
    
    print(f"\n📊 Key Fields for Analysis:")
    key_fields = guidance.get('key_fields_for_analysis', [])
    for field in key_fields:
        available = "✅" if file_columns and field in file_columns else "❓"
        print(f"   {available} {field}")
    
    print(f"\n🛠️ Applicable Analysis Templates:")
    templates = list_applicable_summaries(filename)
    for template in templates:
        template_info = SUMMARY_TEMPLATES[template]
        print(f"   • {template}: {template_info['description']}")
    
    # Show any special analysis notes
    for note_type in ['geographic_aggregation', 'model_validation', 'join_keys', 'vmt_calculation']:
        if note_type in guidance:
            print(f"\n💡 {note_type.replace('_', ' ').title()}: {guidance[note_type]}")

print("🎯 Enhanced validation and analysis recommendation functions loaded!")
print("📚 This documentation will help with:")
print("   • Field validation against expected schemas")
print("   • Analysis recommendations for each file type") 
print("   • Pre-built analysis templates")
print("   • Data quality validation")
print("   • Size and structure expectations")

In [10]:
def identify_ctramp_file_type(filename: str) -> tuple:
    """
    Identify the CTRAMP file type based on filename patterns
    Returns: (file_type_key, expected_schema, description)
    """
    filename_lower = filename.lower()
    
    # Check each known pattern
    for pattern, schema in CTRAMP_FILE_SCHEMAS.items():
        if pattern.replace('_[iteration]', '').replace('[iteration]', '') in filename_lower:
            return pattern, schema, schema['description']
    
    # Check for specific patterns with iterations
    for base_name in ['householdData', 'personData', 'wsLocResults', 'indivTourData', 
                      'indivTripData', 'jointTourData', 'jointTripData', 'indivTripDataResim',
                      'unconstrainedPNRDemand', 'constrainedPNRDemand']:
        if base_name.lower() in filename_lower:
            return base_name, CTRAMP_FILE_SCHEMAS[base_name], CTRAMP_FILE_SCHEMAS[base_name]['description']
    
    return None, None, "Unknown CTRAMP file type"

def validate_schema_against_documentation(file_info: dict) -> dict:
    """
    Validate discovered schema against expected CTRAMP documentation
    """
    filename = file_info['filename']
    file_type_key, expected_schema, description = identify_ctramp_file_type(filename)
    
    validation_result = {
        'filename': filename,
        'identified_type': file_type_key,
        'description': description,
        'validation_status': 'unknown',
        'missing_fields': [],
        'unexpected_fields': [],
        'field_match_percentage': 0,
        'total_expected_fields': 0,
        'total_actual_fields': len(file_info.get('columns', [])),
        'has_utility_fields': False,
        'has_probability_fields': False,
        'appears_to_be_ctramp': False
    }
    
    if file_info.get('error'):
        validation_result['validation_status'] = 'error'
        return validation_result
        
    actual_columns = set(file_info.get('columns', []))
    
    if expected_schema:
        expected_fields = set(expected_schema['expected_fields'])
        validation_result['total_expected_fields'] = len(expected_fields)
        validation_result['appears_to_be_ctramp'] = True
        
        # Find missing and unexpected fields
        validation_result['missing_fields'] = list(expected_fields - actual_columns)
        validation_result['unexpected_fields'] = list(actual_columns - expected_fields)
        
        # Calculate match percentage (only for core fields, utility/prob fields are dynamic)
        core_expected = {f for f in expected_fields if not (f.startswith('util_') or f.startswith('prob_'))}
        matching_fields = len(core_expected & actual_columns)
        if len(core_expected) > 0:
            validation_result['field_match_percentage'] = (matching_fields / len(core_expected)) * 100
        
        # Check for utility and probability fields (mode choice models)
        validation_result['has_utility_fields'] = any(col.startswith('util_') for col in actual_columns)
        validation_result['has_probability_fields'] = any(col.startswith('prob_') for col in actual_columns)
        
        # Determine overall validation status
        if validation_result['field_match_percentage'] >= 80:
            validation_result['validation_status'] = 'excellent'
        elif validation_result['field_match_percentage'] >= 60:
            validation_result['validation_status'] = 'good'
        elif validation_result['field_match_percentage'] >= 40:
            validation_result['validation_status'] = 'partial'
        else:
            validation_result['validation_status'] = 'poor'
    else:
        # Check if it looks like a CTRAMP file based on common fields
        ctramp_indicators = {'hh_id', 'person_id', 'tour_id', 'trip_mode', 'tour_mode', 'mgra'}
        if any(indicator in actual_columns for indicator in ctramp_indicators):
            validation_result['appears_to_be_ctramp'] = True
            validation_result['validation_status'] = 'unrecognized_ctramp'
        else:
            validation_result['validation_status'] = 'non_ctramp'
            
    return validation_result

# Validate all discovered files against CTRAMP documentation
print("🔍 Validating discovered files against CTRAMP documentation...")
validation_results = []

for file_info in schema_results.get('file_schemas', []):
    validation = validate_schema_against_documentation(file_info)
    validation_results.append(validation)

print(f"✅ Validated {len(validation_results)} files")

🔍 Validating discovered files against CTRAMP documentation...
✅ Validated 12 files


In [11]:
# Create comprehensive validation report
def print_validation_summary(validation_results: list):
    """
    Print a comprehensive validation summary with CTRAMP documentation comparison
    """
    print("📊 CTRAMP VALIDATION SUMMARY")
    print("="*80)
    
    # Overall statistics
    total_files = len(validation_results)
    ctramp_files = sum(1 for v in validation_results if v['appears_to_be_ctramp'])
    excellent_matches = sum(1 for v in validation_results if v['validation_status'] == 'excellent')
    good_matches = sum(1 for v in validation_results if v['validation_status'] == 'good')
    
    print(f"📁 Total Files Analyzed: {total_files}")
    print(f"🎯 CTRAMP Files Identified: {ctramp_files}")
    print(f"✅ Excellent Matches (80%+): {excellent_matches}")
    print(f"👍 Good Matches (60-80%): {good_matches}")
    print("-"*80)
    
    # Detailed file-by-file analysis
    for i, validation in enumerate(validation_results, 1):
        status_emoji = {
            'excellent': '✅', 'good': '👍', 'partial': '⚠️', 
            'poor': '❌', 'error': '💥', 'unrecognized_ctramp': '❓', 'non_ctramp': '📄'
        }
        
        emoji = status_emoji.get(validation['validation_status'], '❓')
        print(f"\n{emoji} FILE {i}: {validation['filename']}")
        print(f"   Type: {validation['identified_type'] or 'Unknown'}")
        print(f"   Description: {validation['description']}")
        print(f"   Status: {validation['validation_status'].replace('_', ' ').title()}")
        
        if validation['appears_to_be_ctramp']:
            print(f"   Field Match: {validation['field_match_percentage']:.1f}% ({validation['total_actual_fields']} actual vs {validation['total_expected_fields']} expected)")
            
            if validation['missing_fields']:
                print(f"   Missing Fields ({len(validation['missing_fields'])}): {', '.join(validation['missing_fields'][:5])}{'...' if len(validation['missing_fields']) > 5 else ''}")
            
            if validation['unexpected_fields']:
                print(f"   Extra Fields ({len(validation['unexpected_fields'])}): {', '.join(validation['unexpected_fields'][:5])}{'...' if len(validation['unexpected_fields']) > 5 else ''}")
                
            if validation['has_utility_fields']:
                print("   ✓ Has utility fields (mode choice model)")
            if validation['has_probability_fields']:
                print("   ✓ Has probability fields (mode choice model)")
        
        print("-"*60)

# Print the validation summary
print_validation_summary(validation_results)

📊 CTRAMP VALIDATION SUMMARY
📁 Total Files Analyzed: 12
🎯 CTRAMP Files Identified: 12
✅ Excellent Matches (80%+): 4
👍 Good Matches (60-80%): 8
--------------------------------------------------------------------------------

✅ FILE 1: householdData_1.csv
   Type: householdData
   Description: Household level model results
   Status: Excellent
   Field Match: 100.0% (12 actual vs 4 expected)
   Extra Fields (8): income, autos, workers, automated_vehicles, size...
------------------------------------------------------------

👍 FILE 2: indivTourData_1.csv
   Type: indivTourData
   Description: Individual Tours - one row per individual tour
   Status: Good
   Field Match: 76.0% (54 actual vs 59 expected)
   Missing Fields (6): out_set, in_btap, out_atap, out_btap, in_atap...
   Extra Fields (1): avAvailable
   ✓ Has utility fields (mode choice model)
   ✓ Has probability fields (mode choice model)
------------------------------------------------------------

👍 FILE 3: indivTripData_1.csv
  

In [ ]:
# Create enhanced validation DataFrame with CTRAMP documentation
def create_validation_dataframe(validation_results: list) -> pd.DataFrame:
    """
    Create a detailed validation DataFrame with CTRAMP documentation comparison
    """
    validation_rows = []
    
    for validation in validation_results:
        validation_rows.append({
            'filename': validation['filename'],
            'identified_ctramp_type': validation['identified_type'],
            'file_description': validation['description'], 
            'validation_status': validation['validation_status'],
            'appears_to_be_ctramp': validation['appears_to_be_ctramp'],
            'field_match_percentage': validation['field_match_percentage'],
            'total_actual_fields': validation['total_actual_fields'],
            'total_expected_fields': validation['total_expected_fields'],
            'missing_fields_count': len(validation['missing_fields']),
            'unexpected_fields_count': len(validation['unexpected_fields']),
            'has_utility_fields': validation['has_utility_fields'],
            'has_probability_fields': validation['has_probability_fields'],
            'missing_fields': '; '.join(validation['missing_fields'][:10]),  # Limit for CSV
            'unexpected_fields': '; '.join(validation['unexpected_fields'][:10])  # Limit for CSV
        })
    
    return pd.DataFrame(validation_rows)

# Create enhanced validation DataFrame
validation_df = create_validation_dataframe(validation_results)

print("📊 ENHANCED VALIDATION RESULTS")
print(f"Total files analyzed: {len(validation_df)}")
print(f"CTRAMP files identified: {validation_df['appears_to_be_ctramp'].sum()}")
print("\nValidation Status Distribution:")
print(validation_df['validation_status'].value_counts())

# Show top matches
print(f"\n🏆 TOP SCHEMA MATCHES:")
top_matches = validation_df[validation_df['appears_to_be_ctramp']].nlargest(5, 'field_match_percentage')
for _, row in top_matches.iterrows():
    print(f"   {row['filename']}: {row['field_match_percentage']:.1f}% match ({row['identified_ctramp_type']})")

# Export enhanced validation results
validation_output_file = "ctramp_2023_validation_results.csv"
validation_df.to_csv(validation_output_file, index=False)
print(f"\n✅ Enhanced validation results exported to: {validation_output_file}")

In [ ]:
# Data Quality Checks with CTRAMP Knowledge
def analyze_data_quality_with_ctramp_context(schema_results: dict, validation_results: list):
    """
    Perform data quality analysis using CTRAMP domain knowledge
    """
    print("🔍 CTRAMP DATA QUALITY ANALYSIS")
    print("="*80)
    
    # Find potential issues based on CTRAMP knowledge
    issues_found = []
    
    for i, (file_info, validation) in enumerate(zip(schema_results['file_schemas'], validation_results)):
        if file_info.get('error'):
            continue
            
        filename = file_info['filename'] 
        columns = file_info.get('columns', [])
        file_type = validation['identified_type']
        
        # Check for common CTRAMP data quality issues
        file_issues = []
        
        # 1. Check for required ID fields
        if validation['appears_to_be_ctramp']:
            if 'hh_id' not in columns and file_type not in ['accessibilities.csv']:
                file_issues.append("Missing household ID (hh_id)")
            
            if file_type in ['personData', 'indivTourData', 'indivTripData', 'indivTripDataResim']:
                if 'person_id' not in columns:
                    file_issues.append("Missing person ID (person_id)")
                    
            if file_type in ['indivTourData', 'jointTourData']:
                if 'tour_id' not in columns:
                    file_issues.append("Missing tour ID (tour_id)")
                    
            if file_type in ['indivTripData', 'jointTripData', 'indivTripDataResim']:
                if 'stop_id' not in columns:
                    file_issues.append("Missing stop/trip ID (stop_id)")
        
        # 2. Check for mode and time period fields
        mode_fields = [col for col in columns if 'mode' in col.lower()]
        period_fields = [col for col in columns if 'period' in col.lower()]
        
        if validation['appears_to_be_ctramp'] and file_type in ['indivTourData', 'indivTripData', 'jointTourData', 'jointTripData']:
            if not mode_fields:
                file_issues.append("No mode fields found")
            if not period_fields:
                file_issues.append("No time period fields found")
        
        # 3. Check for utility/probability field consistency
        util_fields = [col for col in columns if col.startswith('util_')]
        prob_fields = [col for col in columns if col.startswith('prob_')]
        
        if util_fields and not prob_fields:
            file_issues.append("Has utility fields but missing probability fields")
        elif prob_fields and not util_fields:
            file_issues.append("Has probability fields but missing utility fields")
        elif util_fields and prob_fields:
            if len(util_fields) != len(prob_fields):
                file_issues.append(f"Utility/probability field count mismatch ({len(util_fields)} vs {len(prob_fields)})")
        
        # 4. Check for geographic fields (MAZ/TAZ)
        geo_fields = [col for col in columns if 'mgra' in col.lower() or 'taz' in col.lower() or 'tap' in col.lower()]
        if validation['appears_to_be_ctramp'] and file_type not in ['householdData', 'personData']:
            if not geo_fields:
                file_issues.append("No geographic fields (MGRA/TAZ/TAP) found")
        
        # 5. Sample size estimation
        if file_info.get('shape'):
            rows = file_info['shape'][0]
            if validation['appears_to_be_ctramp']:
                if file_type in ['householdData'] and rows < 100000:
                    file_issues.append(f"Low household count ({rows:,}) - expected 100K+ for Bay Area")
                elif file_type in ['personData'] and rows < 200000:
                    file_issues.append(f"Low person count ({rows:,}) - expected 200K+ for Bay Area")
                elif file_type in ['indivTripData'] and rows < 1000000:
                    file_issues.append(f"Low trip count ({rows:,}) - expected 1M+ for Bay Area")
        
        if file_issues:
            issues_found.append({
                'filename': filename,
                'file_type': file_type,
                'issues': file_issues,
                'issue_count': len(file_issues)
            })
    
    # Report findings
    if issues_found:
        print(f"⚠️  Found potential issues in {len(issues_found)} files:")
        for issue_info in issues_found:
            print(f"\n📄 {issue_info['filename']} ({issue_info['file_type']})")
            for issue in issue_info['issues']:
                print(f"   • {issue}")
    else:
        print("✅ No major data quality issues detected!")
    
    return issues_found

# Run data quality analysis
quality_issues = analyze_data_quality_with_ctramp_context(schema_results, validation_results)

## 📋 CTRAMP Documentation Summary

### Key File Types Expected:
- **Individual Tours** (`indivTourData_*.csv`): Person-level tours with mode choice utilities
- **Individual Trips** (`indivTripData_*.csv`): Person-level trips with detailed geography 
- **Joint Tours/Trips** (`jointTourData_*.csv`, `jointTripData_*.csv`): Household joint travel
- **Household Data** (`householdData_*.csv`): HH characteristics and model results
- **Person Data** (`personData_*.csv`): Person attributes and choice model results
- **Location Choice** (`wsLocResults_*.csv`): Work/school location model results
- **Auto Ownership** (`aoResults*.csv`): Vehicle ownership model results
- **Accessibility** (`accessibilities.csv`): Zone-level accessibility measures

### Mode Codes (1-17):
1-2: Drive Alone (Free/Pay), 3-8: Shared Ride (2/3+ persons, GP/HOV/Pay), 9-10: Walk/Bike, 
11-14: Transit (Walk/PNR/KNR), 15-17: Taxi/TNC/School Bus

### Time Periods (1-40):
30-minute periods from 3:00 AM to 3:00 AM next day, with Period 1 = 3:00-5:00 AM

### Important Notes:
- Files may have iteration numbers in names (e.g., `_1.csv`, `_2.csv`)
- Tour/trip files have mode choice utilities (`util_1` to `util_17`) and probabilities (`prob_1` to `prob_17`)
- Geographic fields use MGRA (microzones), TAP (transit access points) 
- Sample rates may vary by iteration for memory management
- Joint travel files have one record per trip (not per person)